In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import datetime as dt
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [80]:
# load all csv files into respective dataframes
BTC_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\BTC-USD (3).csv')

AUD_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\AUDUSD=X.csv')
EUR_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\EURUSD=X.csv')
GBP_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\GBPUSD=X.csv')
JPY_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\JPY=X.csv')

SOL_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\SOL-USD.csv')
ETH_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\ETH-USD.csv')
LTC_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\LTC-USD.csv')
DOGE_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\DOGE-USD.csv')

DGS10_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\DGS10 (3).csv')
DTB3_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\DTB3 (1).csv')

USEP_df = pd.read_csv(r'..\\Project_2_BTC_Price_Modeling\csv_files\USEPUINDXD.csv')
GEP_df = pd.read_csv('..\\Project_2_BTC_Price_Modeling\csv_files\GEPUCURRENT.csv')





In [65]:
#Remove High, low, Adj Close and volume column from BTC dataframe

BTC_df = BTC_df.drop(["High","Low","Volume", "Adj Close"], axis=1)

BTC_df.head()


,Date,Open,Close
0,2014-09-15,424.102997,398.821014
1,2014-09-22,399.100006,377.181000
2,2014-09-29,376.928009,320.510010
3,2014-10-06,320.389008,378.549011
4,2014-10-13,377.920990,389.545990


In [66]:
#Create a return rate column, and another column that encodes for positive (1) and negative returns (0) rename to specify BTC data
BTC_df['BTC Return Rate'] = (BTC_df['Close'] - BTC_df['Open']) / BTC_df['Open']

BTC_df['BTC Return Class'] = BTC_df['BTC Return Rate'].apply(lambda x: 1 if x > 0 else 0)

#rename to specify BTC data

BTC_rename_df = BTC_df.rename(columns={'Open':'BTC Open','Close':'BTC Close'})
BTC_rename_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              522 non-null    object 
 1   BTC Open          521 non-null    float64
 2   BTC Close         521 non-null    float64
 3   BTC Return Rate   521 non-null    float64
 4   BTC Return Class  522 non-null    int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 20.5+ KB


In [67]:
#Drop unecessary rows from forex data

AUD_clean_df = AUD_df.drop(["High","Low","Volume", "Adj Close"], axis=1)
EUR_clean_df = EUR_df.drop(["High","Low","Volume", "Adj Close"], axis=1)
GBP_clean_df = GBP_df.drop(["High","Low","Volume", "Adj Close"], axis=1)
JPY_clean_df = JPY_df.drop(["High","Low","Volume", "Adj Close"], axis=1)



In [68]:

#Create return rate columns for FOREX

AUD_clean_df['AUD Return Rate'] = (AUD_clean_df['Close'] - AUD_clean_df['Open']) /AUD_clean_df['Open']
EUR_clean_df['EUR Return Rate'] = (EUR_clean_df['Close'] - EUR_clean_df['Open']) /EUR_clean_df['Open']
GBP_clean_df['GBP Return Rate'] = (GBP_clean_df['Close'] - GBP_clean_df['Open']) /GBP_clean_df['Open']
JPY_clean_df['JPY Return Rate'] = (JPY_clean_df['Close'] - JPY_clean_df['Open']) /JPY_clean_df['Open']

AUD_rename_df = AUD_clean_df.rename(columns={'Open':'AUD Open','Close':'AUD Close'})
EUR_rename_df = EUR_clean_df.rename(columns={'Open':'EUR Open','Close':'EUR Close'})
GBP_rename_df = GBP_clean_df.rename(columns={'Open':'GBP Open','Close':'GBP Close'})
JPY_rename_df = JPY_clean_df.rename(columns={'Open':'JPY Open','Close':'JPY Close'})



In [69]:
#Combine treasury bills dataframe, and divide by 100 to reduce to decimals instead of percentages
combined_treasuries_df = DGS10_df.merge(DTB3_df, on='Date',how='outer')
combined_treasuries_df['DGS10'] = combined_treasuries_df['DGS10']/100
combined_treasuries_df['DTB3'] = combined_treasuries_df['DTB3']/100
combined_treasuries_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    520 non-null    object 
 1   DGS10   520 non-null    float64
 2   DTB3    520 non-null    float64
dtypes: float64(2), object(1)
memory usage: 12.3+ KB


In [70]:
#Merge FOREX data frames

combined_currency_ex_df = AUD_rename_df.merge(EUR_rename_df, on='Date', how='outer').merge(GBP_rename_df, on='Date', how='outer').merge(JPY_rename_df, on='Date', how='outer')
clean_currency_df = combined_currency_ex_df.drop(['AUD Open','AUD Close','EUR Open','EUR Close','GBP Open','GBP Close','JPY Open','JPY Close'], axis=1)
clean_currency_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             522 non-null    object 
 1   AUD Return Rate  522 non-null    float64
 2   EUR Return Rate  522 non-null    float64
 3   GBP Return Rate  522 non-null    float64
 4   JPY Return Rate  522 non-null    float64
dtypes: float64(4), object(1)
memory usage: 20.5+ KB


In [79]:


#Merge Uncertainty index
combined_uncertainty_df =  USEP_df.merge(GEP_df, on = "DATE", how='outer')
combined_uncertainty_rename_df = combined_uncertainty_df.rename(columns={"DATE":'Date'})
combined_uncertainty_rename_df.head()




,Date,USEPUINDXD,GEPUCURRENT
0,2014-09-15,67.894286,NaN
1,2014-09-22,58.347143,NaN
2,2014-09-29,55.678571,NaN
3,2014-10-06,72.465714,NaN
4,2014-10-13,59.967143,NaN


In [81]:
# Function to repeat Global Economic uncertainty values based on days in the month

GEP_df= pd.to_datetime(['DATE'])

def repeat_values_based_on_days(df):
    # Create a new DataFrame to store the repeated values
    repeated_df = pd.DataFrame()
    
    for index, row in df.iterrows():
        # Extract the year and month
        year = row['DATE'].year
        month = row['DATE'].month
        
        # Get the number of days in the month
        num_days = pd.Period(f"{year}-{month}").days_in_month
        
        # Repeat the value for the number of days
        repeated_values = pd.DataFrame({
            'DATE': pd.date_range(start=f'{year}-{month}-01', periods=num_days, freq='D'),
            'GEPUCURRENT': np.repeat(row['GEPUCURRENT'], num_days)
        })
        
        # Append to the repeated_df
        repeated_df = pd.concat([repeated_df, repeated_values])
    
    return repeated_df.reset_index(drop=True)

# Apply the function
result_df = repeat_values_based_on_days(GEP_df)

DateParseError: Unknown datetime string format, unable to parse: DATE, at position 0